In this notebook we're goin to use GraphQL's built-in introspection features to explore what's in the CDS database.  This is not intended to be a full tutorial on introspection, but rather a brief view of how introspection can be used to gain a better understanding of what's in CDS.\
For more information on introspection and GraphQL, visit the GraphQL website: https://graphql.org/learn/  \

This notebook has examples of how to use the CDS API to do common queries and return results that should be usable, and exportable to a Cloud Resource. A few useful links:

    The CDS Data Model: https://dataservice.datacommons.cancer.gov/#/resources
    The CDS GraphQL endpoint: https://dataservice.datacommons.cancer.gov/v1/graphql/
    The GraphiQL interface in CDS (a good place to build queries): https://dataservice.datacommons.cancer.gov/#/graphql

Import a few useful libraries. Requests for carrying out the communications and Pandas for manipulating the returns. We also use parts of IPython to format output from the dataframes to be readable.


In [3]:
import pandas as pd
import requests
from IPython.display import display, Markdown, Latex

In [4]:
cds_graphql_url = "https://dataservice.datacommons.cancer.gov/v1/graphql/"

Since there will be multiple queries, having a simple routine that runs the query and returns the answer as a JSON object will streamline the notebook

In [5]:
def runGraphQLQuery(url,query,variables):
    headers = {"content-type":"application/json"}
    try:
        if variables is None:
            results = requests.post(url = url, json={"query":query, "headers":headers})
        else:
            results = requests.post(url=url, json={"query":query, "headers":headers, "variables":variables})
        results = results.json()
    except requests.exceptions.HTTPError as exception:
        print(exception)
    return results

Let's start by using the __schema introspection query to see what's avaiable in CDS.  This should list the various kinds of queries that we can use.

In [6]:
basic_query = """
{
  __schema{
    types{
      name
      kind
      description
    }
  }
}
"""

In [7]:
cds_fields = runGraphQLQuery(cds_graphql_url, basic_query, None)

In [8]:
field_list = cds_fields['data']['__schema']['types']

Let's take a look at this info to get an idea of what we can do with the CDS GraphQL API

In [9]:
columns = ["Name", "Kind", "Description"]
basic_df = pd.DataFrame(columns = columns)
for entry in field_list:
    if entry['description'] is None:
        description = "None"
    else:
        description = entry['description']
    basic_df.loc[len(basic_df.index)] = [entry['name'],entry['kind'],description]

In [10]:
display(Markdown(basic_df.to_markdown()))

|    | Name                        | Kind         | Description                                                                                                                                                                                           |
|---:|:----------------------------|:-------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | Boolean                     | SCALAR       | Built-in Boolean                                                                                                                                                                                      |
|  1 | FileInfo                    | OBJECT       | None                                                                                                                                                                                                  |
|  2 | FileOverview                | OBJECT       | None                                                                                                                                                                                                  |
|  3 | Float                       | SCALAR       | Built-in Float                                                                                                                                                                                        |
|  4 | GS_About                    | OBJECT       | None                                                                                                                                                                                                  |
|  5 | GS_File                     | OBJECT       | None                                                                                                                                                                                                  |
|  6 | GS_Model                    | OBJECT       | None                                                                                                                                                                                                  |
|  7 | GS_Program                  | OBJECT       | None                                                                                                                                                                                                  |
|  8 | GS_Sample                   | OBJECT       | None                                                                                                                                                                                                  |
|  9 | GS_Study                    | OBJECT       | None                                                                                                                                                                                                  |
| 10 | GS_Subject                  | OBJECT       | None                                                                                                                                                                                                  |
| 11 | GlobalSearchResult          | OBJECT       | None                                                                                                                                                                                                  |
| 12 | GroupCount                  | OBJECT       | None                                                                                                                                                                                                  |
| 13 | GroupCountNeo4j             | OBJECT       | None                                                                                                                                                                                                  |
| 14 | IdsLists                    | OBJECT       | None                                                                                                                                                                                                  |
| 15 | Int                         | SCALAR       | Built-in Int                                                                                                                                                                                          |
| 16 | Mutation                    | OBJECT       | None                                                                                                                                                                                                  |
| 17 | ProgramDetail               | OBJECT       | None                                                                                                                                                                                                  |
| 18 | ProgramDetailStudy          | OBJECT       | None                                                                                                                                                                                                  |
| 19 | ProgramListEntry            | OBJECT       | None                                                                                                                                                                                                  |
| 20 | QueryType                   | OBJECT       | None                                                                                                                                                                                                  |
| 21 | RangeCount                  | OBJECT       | None                                                                                                                                                                                                  |
| 22 | RelationDirection           | ENUM         | None                                                                                                                                                                                                  |
| 23 | SampleOverview              | OBJECT       | None                                                                                                                                                                                                  |
| 24 | SearchResult                | OBJECT       | None                                                                                                                                                                                                  |
| 25 | String                      | SCALAR       | Built-in String                                                                                                                                                                                       |
| 26 | StudyDetail                 | OBJECT       | None                                                                                                                                                                                                  |
| 27 | StudyInfo                   | OBJECT       | None                                                                                                                                                                                                  |
| 28 | SubjectDetail               | OBJECT       | None                                                                                                                                                                                                  |
| 29 | SubjectOverview             | OBJECT       | None                                                                                                                                                                                                  |
| 30 | SubjectResult               | OBJECT       | None                                                                                                                                                                                                  |
| 31 | _FileInfoFilter             | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 32 | _FileInfoOrdering           | ENUM         | None                                                                                                                                                                                                  |
| 33 | _GroupCountNeo4jFilter      | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 34 | _GroupCountNeo4jOrdering    | ENUM         | None                                                                                                                                                                                                  |
| 35 | _IdsListsFilter             | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 36 | _IdsListsOrdering           | ENUM         | None                                                                                                                                                                                                  |
| 37 | _ProgramDetailFilter        | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 38 | _ProgramDetailOrdering      | ENUM         | None                                                                                                                                                                                                  |
| 39 | _ProgramDetailStudyFilter   | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 40 | _ProgramDetailStudyOrdering | ENUM         | None                                                                                                                                                                                                  |
| 41 | _ProgramListEntryFilter     | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 42 | _ProgramListEntryOrdering   | ENUM         | None                                                                                                                                                                                                  |
| 43 | _StudyDetailFilter          | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 44 | _StudyDetailOrdering        | ENUM         | None                                                                                                                                                                                                  |
| 45 | _StudyInfoFilter            | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 46 | _StudyInfoOrdering          | ENUM         | None                                                                                                                                                                                                  |
| 47 | _SubjectDetailFilter        | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 48 | _SubjectDetailOrdering      | ENUM         | None                                                                                                                                                                                                  |
| 49 | __Directive                 | OBJECT       | None                                                                                                                                                                                                  |
| 50 | __DirectiveLocation         | ENUM         | An enum describing valid locations where a directive can be placed                                                                                                                                    |
| 51 | __EnumValue                 | OBJECT       | None                                                                                                                                                                                                  |
| 52 | __Field                     | OBJECT       | None                                                                                                                                                                                                  |
| 53 | __InputValue                | OBJECT       | None                                                                                                                                                                                                  |
| 54 | __Schema                    | OBJECT       | A GraphQL Introspection defines the capabilities of a GraphQL server. It exposes all available types and directives on the server, the entry points for query, mutation, and subscription operations. |
| 55 | __Type                      | OBJECT       | None                                                                                                                                                                                                  |
| 56 | __TypeKind                  | ENUM         | An enum describing what kind of type a given __Type is                                                                                                                                                |
| 57 | _diagnosisFilter            | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 58 | _diagnosisOrdering          | ENUM         | None                                                                                                                                                                                                  |
| 59 | _fileFilter                 | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 60 | _fileOrdering               | ENUM         | None                                                                                                                                                                                                  |
| 61 | _genomic_infoFilter         | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 62 | _genomic_infoOrdering       | ENUM         | None                                                                                                                                                                                                  |
| 63 | _participantFilter          | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 64 | _participantOrdering        | ENUM         | None                                                                                                                                                                                                  |
| 65 | _programFilter              | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 66 | _programOrdering            | ENUM         | None                                                                                                                                                                                                  |
| 67 | _sampleFilter               | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 68 | _sampleOrdering             | ENUM         | None                                                                                                                                                                                                  |
| 69 | _specimenFilter             | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 70 | _specimenOrdering           | ENUM         | None                                                                                                                                                                                                  |
| 71 | _studyFilter                | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 72 | _studyOrdering              | ENUM         | None                                                                                                                                                                                                  |
| 73 | _treatmentFilter            | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 74 | _treatmentOrdering          | ENUM         | None                                                                                                                                                                                                  |
| 75 | diagnosis                   | OBJECT       | None                                                                                                                                                                                                  |
| 76 | file                        | OBJECT       | None                                                                                                                                                                                                  |
| 77 | genomic_info                | OBJECT       | None                                                                                                                                                                                                  |
| 78 | participant                 | OBJECT       | None                                                                                                                                                                                                  |
| 79 | program                     | OBJECT       | None                                                                                                                                                                                                  |
| 80 | sample                      | OBJECT       | None                                                                                                                                                                                                  |
| 81 | specimen                    | OBJECT       | None                                                                                                                                                                                                  |
| 82 | study                       | OBJECT       | None                                                                                                                                                                                                  |
| 83 | treatment                   | OBJECT       | None                                                                                                                                                                                                  |

There's quite a lot of ways to query CDS, the one's to pay attention to are those with the OBJECT label.  They can also take an argument (we'll get into that later) to narrow down what gets returned.

Another introspection query that's worth looking through involved the queryType.

In [11]:
query_type_query = """
{
  __schema{
    queryType{
      fields{
        name
        description
      }
    }
  }
}
"""

In [12]:
query_type_results = runGraphQLQuery(cds_graphql_url, query_type_query, None)

In [13]:
query_type_list = query_type_results['data']['__schema']['queryType']['fields']

In [14]:
columns = ["Name", "Description"]
query_type_df = pd.DataFrame(columns=columns)
for entry in query_type_list:
    #print(entry)
    if entry['description'] is None:
        description = "None"
    else:
        description = entry['description']
    query_type_df.loc[len(query_type_df.index)] = [entry['name'],description]

In [15]:
display(Markdown(query_type_df.to_markdown()))

|    | Name                 | Description                                      |
|---:|:---------------------|:-------------------------------------------------|
|  0 | diagnosis            | None                                             |
|  1 | file                 | None                                             |
|  2 | genomic_info         | None                                             |
|  3 | participant          | None                                             |
|  4 | program              | None                                             |
|  5 | sample               | None                                             |
|  6 | specimen             | None                                             |
|  7 | study                | None                                             |
|  8 | treatment            | None                                             |
|  9 | schemaVersion        | None                                             |
| 10 | numberOfStudies      | None                                             |
| 11 | numberOfSubjects     | None                                             |
| 12 | numberOfSamples      | None                                             |
| 13 | numberOfFiles        | None                                             |
| 14 | numberOfDiseaseSites | None                                             |
| 15 | idsLists             | Get lists of all subject ids, used by Local Find |
| 16 | studyDetail          | Study Detail Page                                |
| 17 | subjectDetail        | Subject Detail Page                              |
| 18 | samplesForSubjectId  | None                                             |
| 19 | studyList            | None                                             |
| 20 | programDetail        | None                                             |
| 21 | programList          | None                                             |
| 22 | fileInfo             | None                                             |
| 23 | studyInfo            | None                                             |
| 24 | programDetailStudy   | None                                             |
| 25 | programListEntry     | None                                             |
| 26 | groupCountNeo4j      | None                                             |
| 27 | searchSubjects       | None                                             |
| 28 | subjectOverview      | None                                             |
| 29 | sampleOverview       | None                                             |
| 30 | fileOverview         | None                                             |
| 31 | fileIDsFromList      | None                                             |
| 32 | findSubjectIdsInList | None                                             |
| 33 | globalSearch         | None                                             |

These are the fields that generally contain data and could be of use when conducting a search for data

Lastly, it's worth understanding what is contained in the OBJECT fields from the first query.  To do this, we'll build a query that has a variable built in so we can run it for each field

In [16]:
field_query = """
query cdsFields ($fieldname:String!) {
  __type(name: $fieldname){
    fields{
      name
    }
  }
}
"""

Now loop throught all the fields and build a structure that contains each enty has a listing of it's fields

In [17]:
final_data = {}
for entry in field_list:
    templist = []
    variables = {"fieldname": entry['name']}
    results = runGraphQLQuery(cds_graphql_url, field_query, variables)
    if results['data']['__type']['fields'] is not None:
            for item in results['data']['__type']['fields']:
                for key, name in item.items():
                    templist.append(name)
    final_data[entry['name']] = templist

In [18]:
columns = ['Query Fields', 'Associated Fields']
query_df = pd.DataFrame(columns = columns)
for key, value in final_data.items():
    query_df.loc[len(query_df.index)] = [key, value]


In [19]:
display(Markdown(query_df.to_markdown()))

|    | Query Fields                | Associated Fields                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
|---:|:----------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | Boolean                     | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|  1 | FileInfo                    | ['subject_id', 'file_name', 'file_type', 'association', 'file_description', 'file_size', 'file_id', 'md5sum']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|  2 | FileOverview                | ['study_acronym', 'accesses', 'phs_accession', 'subject_id', 'sample_id', 'experimental_strategy', 'gender', 'analyte_type', 'is_tumor', 'file_name', 'file_type', 'file_size', 'file_id', 'md5sum']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  3 | Float                       | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|  4 | GS_About                    | ['page', 'title', 'type', 'text']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
|  5 | GS_File                     | ['type', 'subject_id', 'sample_id', 'file_id', 'file_name', 'file_type', 'accesses', 'acl', 'experimental_strategies', 'instrument_models', 'library_layouts', 'library_selections', 'library_sources', 'library_strategies', 'platforms', 'reference_genome_assemblies', 'sites']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|  6 | GS_Model                    | ['type', 'node_name', 'property_name', 'property_description', 'property_required', 'property_type', 'value', 'highlight']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
|  7 | GS_Program                  | ['program_name', 'program_short_description', 'program_full_description', 'program_external_url', 'program_sort_order', 'type']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|  8 | GS_Sample                   | ['type', 'sample_id', 'is_tumor', 'analyte_type']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
|  9 | GS_Study                    | ['type', 'phs_accession', 'study_data_types', 'study_name', 'study_code']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| 10 | GS_Subject                  | ['type', 'study', 'subject_id', 'site', 'gender']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| 11 | GlobalSearchResult          | ['study_count', 'studies', 'subject_count', 'subjects', 'sample_count', 'samples', 'file_count', 'files', 'program_count', 'programs', 'about_count', 'about_page', 'model_count', 'model']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 12 | GroupCount                  | ['group', 'subjects']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 13 | GroupCountNeo4j             | ['group', 'subjects']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 14 | IdsLists                    | ['subjectIds']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
| 15 | Int                         | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 16 | Mutation                    | ['creatediagnosis', 'createfile', 'creategenomic_info', 'createparticipant', 'createprogram', 'createsample', 'createspecimen', 'createstudy', 'createtreatment', 'createFileInfo', 'createSubjectDetail', 'createStudyDetail', 'createStudyInfo', 'createIdsLists', 'createProgramDetail', 'createProgramDetailStudy', 'createProgramListEntry', 'createGroupCountNeo4j']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| 17 | ProgramDetail               | ['program', 'program_name', 'program_url', 'program_short_description', 'num_studies', 'num_participants', 'num_files', 'num_samples', 'num_disease_sites', 'study_participants', 'studies']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| 18 | ProgramDetailStudy          | ['accession', 'study_access', 'study_name', 'study_description', 'short_description', 'num_participants', 'num_samples']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| 19 | ProgramListEntry            | ['acronym', 'name', 'website', 'num_studies']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| 20 | QueryType                   | ['diagnosis', 'file', 'genomic_info', 'participant', 'program', 'sample', 'specimen', 'study', 'treatment', 'schemaVersion', 'numberOfStudies', 'numberOfSubjects', 'numberOfSamples', 'numberOfFiles', 'numberOfDiseaseSites', 'idsLists', 'studyDetail', 'subjectDetail', 'samplesForSubjectId', 'studyList', 'programDetail', 'programList', 'fileInfo', 'studyInfo', 'programDetailStudy', 'programListEntry', 'groupCountNeo4j', 'searchSubjects', 'subjectOverview', 'sampleOverview', 'fileOverview', 'fileIDsFromList', 'findSubjectIdsInList', 'globalSearch']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| 21 | RangeCount                  | ['lowerBound', 'upperBound', 'subjects']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| 22 | RelationDirection           | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 23 | SampleOverview              | ['study_acronym', 'phs_accession', 'subject_id', 'sample_id', 'is_tumor', 'analyte_type', 'files']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 24 | SearchResult                | ['numberOfStudies', 'numberOfSubjects', 'numberOfSamples', 'numberOfFiles', 'numberOfDiseaseSites', 'subjectCountByStudy', 'subjectCountByExperimentalStrategy', 'subjectCountByAccess', 'subjectCountByGender', 'subjectCountByIsTumor', 'subjectCountByAnalyteType', 'subjectCountByFileType', 'subjectCountByDiseaseSite', 'subjectCountByLibraryStrategy', 'subjectCountByLibrarySource', 'subjectCountByLibrarySelection', 'subjectCountByLibraryLayout', 'subjectCountByPlatform', 'subjectCountByInstrumentModel', 'subjectCountByReferenceGenomeAssembly', 'subjectCountByPrimaryDiagnosis', 'subjectCountByPhsAccession', 'subjectCountByStudyDataType', 'subjectCountByAcl', 'filterSubjectCountByStudy', 'filterSubjectCountByExperimentalStrategy', 'filterSubjectCountByAccess', 'filterSubjectCountByGender', 'filterSubjectCountByIsTumor', 'filterSubjectCountByFileType', 'filterSubjectCountByAnalyteType', 'filterSubjectCountByDiseaseSite', 'filterSubjectCountByLibraryStrategy', 'filterSubjectCountByLibrarySource', 'filterSubjectCountByLibrarySelection', 'filterSubjectCountByLibraryLayout', 'filterSubjectCountByPlatform', 'filterSubjectCountByInstrumentModel', 'filterSubjectCountByReferenceGenomeAssembly', 'filterSubjectCountByPrimaryDiagnosis', 'filterSubjectCountByPhsAccession', 'filterSubjectCountByStudyDataType', 'filterSubjectCountByAcl', 'filterSubjectCountByNumberOfStudyParticipants', 'filterSubjectCountByNumberOfStudySamples'] |
| 25 | String                      | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 26 | StudyDetail                 | ['study_name', 'phs_accession', 'study_acronym', 'study_description', 'data_types', 'study_external_url', 'numberOfSubjects', 'numberOfSamples', 'numberOfDiseaseSites', 'numberOfFiles']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| 27 | StudyInfo                   | ['study_name', 'phs_accession', 'data_type', 'numberOfSubjects', 'numberOfFiles', 'study_access']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| 28 | SubjectDetail               | ['subject_id', 'study_acronym', 'study_name', 'gender', 'site', 'files', 'samples', 'numberOfSamples', 'numberOfDiseaseSites', 'numberOfFiles']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| 29 | SubjectOverview             | ['subject_id', 'study_acronym', 'phs_accession', 'gender', 'site', 'samples', 'files']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| 30 | SubjectResult               | ['subject_id', 'phs_accession']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| 31 | _FileInfoFilter             | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 32 | _FileInfoOrdering           | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 33 | _GroupCountNeo4jFilter      | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 34 | _GroupCountNeo4jOrdering    | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 35 | _IdsListsFilter             | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 36 | _IdsListsOrdering           | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 37 | _ProgramDetailFilter        | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 38 | _ProgramDetailOrdering      | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 39 | _ProgramDetailStudyFilter   | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 40 | _ProgramDetailStudyOrdering | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 41 | _ProgramListEntryFilter     | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 42 | _ProgramListEntryOrdering   | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 43 | _StudyDetailFilter          | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 44 | _StudyDetailOrdering        | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 45 | _StudyInfoFilter            | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 46 | _StudyInfoOrdering          | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 47 | _SubjectDetailFilter        | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 48 | _SubjectDetailOrdering      | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 49 | __Directive                 | ['name', 'description', 'isRepeatable', 'locations', 'args']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| 50 | __DirectiveLocation         | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 51 | __EnumValue                 | ['name', 'description', 'isDeprecated', 'deprecationReason']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| 52 | __Field                     | ['name', 'description', 'args', 'type', 'isDeprecated', 'deprecationReason']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| 53 | __InputValue                | ['name', 'description', 'type', 'defaultValue', 'isDeprecated', 'deprecationReason']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 54 | __Schema                    | ['description', 'types', 'queryType', 'mutationType', 'directives', 'subscriptionType']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| 55 | __Type                      | ['kind', 'name', 'description', 'fields', 'interfaces', 'possibleTypes', 'enumValues', 'inputFields', 'ofType', 'specifiedByUrl']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| 56 | __TypeKind                  | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 57 | _diagnosisFilter            | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 58 | _diagnosisOrdering          | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 59 | _fileFilter                 | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 60 | _fileOrdering               | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 61 | _genomic_infoFilter         | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 62 | _genomic_infoOrdering       | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 63 | _participantFilter          | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 64 | _participantOrdering        | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 65 | _programFilter              | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 66 | _programOrdering            | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 67 | _sampleFilter               | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 68 | _sampleOrdering             | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 69 | _specimenFilter             | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 70 | _specimenOrdering           | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 71 | _studyFilter                | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 72 | _studyOrdering              | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 73 | _treatmentFilter            | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 74 | _treatmentOrdering          | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 75 | diagnosis                   | ['diagnosis_id', 'disease_type', 'vital_status', 'primary_diagnosis', 'primary_site', 'age_at_diagnosis', 'tumor_grade', 'tumor_stage_clinical_m', 'tumor_stage_clinical_n', 'tumor_stage_clinical_t', 'morphology', 'incidence_type', 'progression_or_recurrence', 'days_to_recurrence', 'days_to_last_followup', 'last_known_disease_status', 'days_to_last_known_status', 'participant']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 76 | file                        | ['file_id', 'file_name', 'file_type', 'file_description', 'file_size', 'md5sum', 'file_url_in_cds', 'experimental_strategy_and_data_subtypes', 'study', 'samples', 'genomic_info']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 77 | genomic_info                | ['library_id', 'bases', 'number_of_reads', 'avg_read_length', 'coverage', 'reference_genome_assembly', 'custom_assembly_fasta_file_for_alignment', 'design_description', 'library_strategy', 'library_layout', 'library_source', 'library_selection', 'platform', 'instrument_model', 'sequence_alignment_software', 'files', 'samples', 'file']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| 78 | participant                 | ['participant_id', 'race', 'gender', 'ethnicity', 'dbGaP_subject_id', 'study', 'diagnoses', 'specimens', 'samples']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| 79 | program                     | ['program_name', 'program_acronym', 'program_short_description', 'program_full_description', 'program_external_url', 'program_sort_order', 'studies']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 80 | sample                      | ['sample_id', 'sample_type', 'sample_tumor_status', 'sample_anatomic_site', 'sample_age_at_collection', 'derived_from_specimen', 'biosample_accession', 'participant', 'specimen', 'files', 'genomic_info']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 81 | specimen                    | ['specimen_id', 'participant', 'samples']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| 82 | study                       | ['study_name', 'study_acronym', 'study_description', 'short_description', 'study_external_url', 'primary_investigator_name', 'primary_investigator_email', 'co_investigator_name', 'co_investigator_email', 'phs_accession', 'bioproject_accession', 'index_date', 'cds_requestor', 'funding_agency', 'funding_source_program_name', 'grant_id', 'clinical_trial_system', 'clinical_trial_identifier', 'clinical_trial_arm', 'organism_species', 'adult_or_childhood_study', 'data_types', 'file_types', 'data_access_level', 'cds_primary_bucket', 'cds_secondary_bucket', 'cds_tertiary_bucket', 'number_of_participants', 'number_of_samples', 'study_data_types', 'file_types_and_format', 'size_of_data_being_uploaded', 'size_of_data_being_uploaded_unit', 'size_of_data_being_uploaded_original', 'size_of_data_being_uploaded_original_unit', 'acl', 'study_access', 'program', 'participants', 'files']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| 83 | treatment                   | ['treatment_id', 'treatment_type', 'treatment_outcome', 'days_to_treatment', 'therapeutic_agents']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |

At this point, we've used GraphQL introspection to get a handle on what's in the database and how we might access it.  In our next notebook, we'll build on this knowledge to search for files that we can use in analyses.